In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('..')
import numpy as np
from numpy.random import random as rand

import networkqit as nq
import networkx as nx

from networkqit import graph_laplacian as GL

import scipy.optimize
import scipy.linalg

import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')

#import sympy as sp
#import sympyprinting as spp
#from sympy import init_printing

In [4]:
# THIS IS THE MATHEMATICA VERSION USING MPMATH FOR HIGHER ACCURACY
import mpmath as mp
def Gamma(a,z0,z1): # This defined function has the same behaviour as Mathematica Gamma[a,z0,z1]
    from mpmath import gammainc
    return gammainc(a,a=z0,b=z1,regularized=False)

def scipyGamma(a,z0,z1):
    return (scipy.special.gammainc(a,z1)-scipy.special.gammainc(a,z0))*scipy.special.gamma(a)

Gamma(3.5j,0,10)

mpc(real='0.0054741038497352953', imag='0.00040984640431357779')

In [5]:
def mpmathfeq(n,pin,pout,t,z):
    #print(n*(pin + pout)*t - z,0.0,-(n*pin)*0.5)
    Eq = t + (2.0**(n*(pin + pout)*t - z)*(-(n*pin))**(-(n*(pin + pout)*t) + z)*Gamma(n*(pin + pout)*t - z,0,-(n*pin)/2.))/mp.exp((n*pin)/2.)
    print('t=',t,'Eq=',(float(Eq.real),float(Eq.imag)))
    return (float(Eq.real),float(Eq.imag))

def scipyfeq(n,pin,pout,t,z):
    print(n*(pin + pout)*t - z,0.0,-(n*pin)*0.5)
    Eq = t + (2.0**(n*(pin + pout)*t - z)*(-(n*pin))**(-(n*(pin + pout)*t) + z)*scipyGamma(n*(pin + pout)*t - z,0.0,-(n*pin)*0.5))/np.exp((n*pin)/2.)
    return (float(Eq.real),float(Eq.imag))

In [ ]:
scipy.optimize.fsolve(lambda t : mpmathfeq(50,0.5,0.2,t[0]+1j*t[1],1),(0.05,0.05))

In [ ]:
for z in np.linspace(0,20,100):
    print(z,scipy.optimize.fsolve(lambda x : mpmathfeq(50,0.5,0.2,x[0]+1j*x[1],z),np.random.random([2,1])*0.01)[1])